### Setup variables, paths, and create artifact registry

In [1]:
!pwd

/home/jupyter/vertex-examples/custom-nb-container


In [12]:
REPO_NAME="workbench"
REGION="us-central1"
PROJECT="hybrid-vertex" # TODO: update with your project_id
IMAGE_ID="python-3.9"
DERIVED_IMAGE_NAME=f"{REGION}-docker.pkg.dev/{PROJECT}/{REPO_NAME}/derived-{IMAGE_ID}"
SCRATCH_IMAGE_NAME=f"{REGION}-docker.pkg.dev/{PROJECT}/{REPO_NAME}/scratch-{IMAGE_ID}"

DERIVED_CONTAINER="derived_container"
SCRATCH_CONTAINER="scratch_container"

# !mkdir {DERIVED_CONTAINER} {SCRATCH_CONTAINER}

# !gcloud beta artifacts repositories create {REPO_NAME} \
    # --repository-format=docker \
    # --location=$REGION

In [13]:
# DERIVED_IMAGE_NAME

In [14]:
# !gcloud config set project $PROJECT

In [15]:
# !gcloud projects add-iam-policy-binding hybrid-vertex --member=serviceAccount:934903580331-compute@developer.gserviceaccount.com --role=roles/artifactregistry.admin

In [16]:
# !gcloud auth configure-docker us-central1-docker.pkg.dev --quiet

### Create Docker image derived from Deeplearning containers

In [17]:
%%writefile {DERIVED_CONTAINER}/Dockerfile
FROM gcr.io/deeplearning-platform-release/tf-gpu:latest
RUN conda create -n python-3.9
RUN conda install -n python-3.9 ipykernel python=3.9 
#SHELL ["/bin/bash", "-c"]
#RUN source /opt/conda/bin/activate /opt/conda/envs/python-3.9 && python -m ipykernel install --user --name=python-3.9 && conda deactivate


Overwriting derived_container/Dockerfile


In [18]:
!docker build . -f $DERIVED_CONTAINER/Dockerfile -t $DERIVED_IMAGE_NAME
!docker push $DERIVED_IMAGE_NAME

Sending build context to Docker daemon  291.8kB
Step 1/3 : FROM gcr.io/deeplearning-platform-release/tf-gpu:latest
 ---> fe0d6c3b8ae6
Step 2/3 : RUN conda create -n python-3.9
 ---> Using cache
 ---> 3b53be1a7c32
Step 3/3 : RUN conda install -n python-3.9 ipykernel python=3.9
 ---> Using cache
 ---> 93ddac834736
Successfully built 93ddac834736
Successfully tagged us-central1-docker.pkg.dev/hybrid-vertex/workbench/derived-python-3.9:latest
Using default tag: latest
The push refers to repository [us-central1-docker.pkg.dev/hybrid-vertex/workbench/derived-python-3.9]

73b7519a: Preparing 
38e52fa8: Preparing 
e8a07179: Preparing 
36f45000: Preparing 
7e28d022: Preparing 
da7d339d: Preparing 
8cec974c: Preparing 
3ab67bd6: Preparing 
29a172ca: Preparing 
6c228da9: Preparing 
2d5f02a3: Preparing 
b8e80bc1: Preparing 
d64e4327: Preparing 
e555f78b: Preparing 
9a89d05b: Preparing 
a28a84fa: Preparing 
28ff1e27: Preparing 
02559779: Preparing 
fb637019: Preparing 
d6bc104f: Preparing 
4e7fb8d6

### Docker Image from scratch

In [19]:
%%writefile {SCRATCH_CONTAINER}/jupyter_notebook_config.py
c.NotebookApp.ip = '*'
c.NotebookApp.token = ''
c.NotebookApp.password = ''
c.NotebookApp.open_browser = False
c.NotebookApp.port = 8080
c.NotebookApp.terminado_settings = {'shell_command': ['/bin/bash']}
c.NotebookApp.allow_origin_pat = (
'(^https://8080-dot-[0-9]+-dot-devshell\.appspot\.com$)|'
'(^https://colab\.research\.google\.com$)|'
'((https?://)?[0-9a-z]+-dot-(?:us|asia|europe|northamerica|southamerica)-?[0-9a-z]+\.notebooks\.googleusercontent.com)')
c.NotebookApp.allow_remote_access = True
c.NotebookApp.disable_check_xsrf = False

Overwriting scratch_container/jupyter_notebook_config.py


In [20]:
%%writefile {SCRATCH_CONTAINER}/Dockerfile
FROM ubuntu:20.04

## for apt to be noninteractive
ENV DEBIAN_FRONTEND noninteractive
ENV DEBCONF_NONINTERACTIVE_SEEN true

RUN apt-get update && apt-get install -y \
    python3.9 \
    python3-pip

# Use this to set the default python to 3.9
# RUN sudo update-alternatives  --set python /usr/bin/python3.9
RUN update-alternatives --install /usr/bin/python3 python3 /usr/bin/python3.9 1
RUN update-alternatives --set python3 /usr/bin/python3.9

RUN pip install jupyter -U && pip install jupyterlab

# RUN python3.9 -m pip install ipykernel && python3.9 -m ipykernel install --user --name "python-3.9"

EXPOSE 8080
             
RUN mkdir /root/.jupyter
             
COPY scratch_container/jupyter_notebook_config.py /root/.jupyter

ENV pwd=""

ENTRYPOINT exec jupyter-lab --ip=0.0.0.0 --port=8080 --no-browser --allow-root --ServerApp.allow_origin="*" --NotebookApp.token="$pwd" --NotebookApp.password="$pwd"

Overwriting scratch_container/Dockerfile


In [21]:
!docker build . -f $SCRATCH_CONTAINER/Dockerfile -t $SCRATCH_IMAGE_NAME
!docker push $SCRATCH_IMAGE_NAME

Sending build context to Docker daemon  186.9kB
Step 1/12 : FROM ubuntu:20.04
 ---> 20fffa419e3a
Step 2/12 : ENV DEBIAN_FRONTEND noninteractive
 ---> Using cache
 ---> b225aa69bffd
Step 3/12 : ENV DEBCONF_NONINTERACTIVE_SEEN true
 ---> Using cache
 ---> 2c16cd0d1d3c
Step 4/12 : RUN apt-get update && apt-get install -y     python3.9     python3-pip
 ---> Using cache
 ---> c62fb21fc387
Step 5/12 : RUN update-alternatives --install /usr/bin/python3 python3 /usr/bin/python3.9 1
 ---> Running in d223d8303c53
update-alternatives: using /usr/bin/python3.9 to provide /usr/bin/python3 (python3) in auto mode
Removing intermediate container d223d8303c53
 ---> 2fa368542797
Step 6/12 : RUN update-alternatives --set python3 /usr/bin/python3.9
 ---> Running in a8d51fc5f03d
Removing intermediate container a8d51fc5f03d
 ---> 1e6a2b94fdf1
Step 7/12 : RUN pip install jupyter -U && pip install jupyterlab
 ---> Running in 7f42999ec7c1
  Created wheel for json5: filename=json5-0.9.8-py2.py3-none-any.whl siz